<a href="https://colab.research.google.com/github/HIJADI/wildlife-img-classification/blob/main/speciesnet_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example speciesnet usage

For automatic object detection and animal classification in camera trap images.

## Input images directory and output

In [16]:
# Example images are under my google drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
IMAGES_DIR =  "/content/drive/MyDrive/SFCT_images"
OUTPUT_JSON = "/content/drive/MyDrive/SFCT_images/speciesnet.json"
IMAGES_DIR_SPECIESNET = "/content/drive/MyDrive/SFCT_images_speciesnet"

!mkdir -p {IMAGES_DIR_SPECIESNET}
!rm -f {OUTPUT_JSON}


In [6]:
!ls -al {IMAGES_DIR}


total 21500
-rw------- 1 root root    14597 Nov  7 17:57 speciesnet.json
-rw------- 1 root root   497614 Nov  6 22:21 STC_0006.JPG
-rw------- 1 root root   616809 Nov  6 22:21 STC_0010.JPG
-rw------- 1 root root 14625136 Nov  6 22:39 STC_0107.AVI
-rw------- 1 root root   663882 Nov  6 22:21 STC_0113.JPG
-rw------- 1 root root   649547 Nov  6 22:21 STC_0117.JPG
-rw------- 1 root root   660920 Nov  6 22:21 STC_0181.JPG
-rw------- 1 root root   784521 Nov  6 22:21 STC_0187.JPG
-rw------- 1 root root   634942 Nov  6 22:21 STC_0270.JPG
-rw------- 1 root root   775024 Nov  6 22:21 STC_0312.JPG
-rw------- 1 root root   670597 Nov  6 22:21 STC_0355.JPG
-rw------- 1 root root   631210 Nov  6 22:21 STC_0482.JPG
-rw------- 1 root root   788656 Nov  6 22:21 STC_0489.JPG


## Install needed packages

In [ ]:
# speciesnet downloads MegaDetector and classification models
# kaggle:google/speciesnet/pyTorch/v4.0.1a etc
!pip install --upgrade speciesnet[notebooks]


In [ ]:
# install megadetector-utils for annotated output images with bboxes
!pip install megadetector-utils


In [3]:
!python3 -m speciesnet.scripts.run_model --help


Script to run the SpeciesNet model.

Provides a command-line interface to execute the SpeciesNet model on various inputs. It
uses flags for specifying input, output, and run options, allowing the user to run the
model in different modes.

flags:

/usr/local/lib/python3.12/dist-packages/speciesnet/scripts/run_model.py:
  --admin1_region: First-level administrative division (in ISO 3166-2 format,
    e.g. 'CA') to enforce on all instances.
  --batch_size: Batch size for classifier inference.
    (default: '8')
    (an integer)
  --[no]bypass_prompts: Whether to bypass confirmation prompts when expected
    files aren't supplied, or unexpected files are supplied. --bypass_prompts
    bypasses prompts, --nobypass_prompts (default) does not.
    (default: 'false')
  --classifications_json: Input JSON file with classifications from previous
    runs.
  --[no]classifier_only: Run only the classifier component. --classifier_only
    enables classifier-only mode, --noclassifier_only (default) d

## Run the object detection and classification pipeline

In [23]:
# run the detector + classification pipeline on the images
# !python3 -m speciesnet.scripts.run_model --folders {IMAGES_DIR} --predictions_json {OUTPUT_JSON}  --country USA
cmd = f"python3 -m speciesnet.scripts.run_model --folders {IMAGES_DIR} --predictions_json {OUTPUT_JSON}  --country USA"
!{cmd}


I1107 22:19:44.701361 139400662003712 classifier.py:109] Loaded SpeciesNetClassifier in 1 second on CPU.
I1107 22:19:45.825698 139400662003712 detector.py:99] Loaded SpeciesNetDetector in 1.12 seconds on CPU.
I1107 22:19:46.473814 139400662003712 ensemble.py:71] Loaded SpeciesNetEnsemble in 0.65 seconds.
Detector preprocess   :   0% 0/11 [00:00<?, ?it/s]
Detector predict      :   0% 0/11 [00:00<?, ?it/s]

Classifier preprocess :   0% 0/11 [00:00<?, ?it/s]


Classifier predict    :   0% 0/2 [00:00<?, ?it/s]



Detector preprocess   :   9% 1/11 [00:00<00:06,  1.45it/s]



Detector preprocess   :  18% 2/11 [00:01<00:04,  1.97it/s]
Detector predict      :   9% 1/11 [00:12<02:08, 12.90s/it]

Classifier preprocess :   9% 1/11 [00:13<02:15, 13.60s/it]



Detector preprocess   :  27% 3/11 [00:13<00:49,  6.16s/it]
Detector predict      :  18% 2/11 [00:25<01:52, 12.51s/it]

Classifier preprocess :  18% 2/11 [00:25<01:52, 12.51s/it]



Detector preprocess   :  36% 4/11 [00:25<00:58,  8.35s/it]
De

## Generate annotated output images

In [24]:
# generate annotated output images
cmd = f"python -m megadetector.visualization.visualize_detector_output  {OUTPUT_JSON} {IMAGES_DIR_SPECIESNET}"
!mkdir -p {PREVIEW_DIR}
!{cmd}


This appears to be a SpeciesNet output file, converting to MD format
Detector output file contains 11 entries.
Rendering detections above a confidence threshold of 0.15
100% 11/11 [00:04<00:00,  2.27it/s]
Skipped 0 failed images (of 11)
Skipped 0 missing images (of 11)
Skipped 0 below-threshold images (of 11)
Rendered detection results to /content/drive/MyDrive/SFCT_images_speciesnet


## Load json classification results

In [25]:
import pandas as pd
import json


In [26]:
# read its json predictions and make a pandas dataframe for it
with open(OUTPUT_JOSN, 'r') as f:
  data = json.load(f)
num_images = len(data['predictions'])
print(f"Images classified: {num_images}")


Images classified: 11


In [28]:
# put predictions in a dataframe
df = pd.json_normalize(data['predictions'])
df


,filepath,country,detections,prediction,prediction_score,prediction_source,model_version,classifications.classes,classifications.scores
0,/content/drive/MyDrive/SFCT_images/STC_0006.JPG,USA,"[{'category': '1', 'label': 'animal', 'conf': ...",aaf3b049-36e6-46dd-9a07-8a580e9618b7;mammalia;...,0.9947,classifier,4.0.1a,[aaf3b049-36e6-46dd-9a07-8a580e9618b7;mammalia...,"[0.9947, 0.0012, 0.0007, 0.0004, 0.0004]"
1,/content/drive/MyDrive/SFCT_images/STC_0010.JPG,USA,"[{'category': '1', 'label': 'animal', 'conf': ...",b1352069-a39c-4a84-a949-60044271c0c1;aves;;;;;...,0.8680,classifier,4.0.1a,[b1352069-a39c-4a84-a949-60044271c0c1;aves;;;;...,"[0.868, 0.029, 0.0125, 0.0094, 0.0062]"
2,/content/drive/MyDrive/SFCT_images/STC_0113.JPG,USA,"[{'category': '1', 'label': 'animal', 'conf': ...",aaf3b049-36e6-46dd-9a07-8a580e9618b7;mammalia;...,0.9830,classifier,4.0.1a,[aaf3b049-36e6-46dd-9a07-8a580e9618b7;mammalia...,"[0.983, 0.0029, 0.0025, 0.0022, 0.0012]"
3,/content/drive/MyDrive/SFCT_images/STC_0117.JPG,USA,"[{'category': '1', 'label': 'animal', 'conf': ...",1f689929-883d-4dae-958c-3d57ab5b6c16;;;;;;animal,0.9022,detector,4.0.1a,[ffe603fc-4920-4302-96d0-65f708bd5b2d;mammalia...,"[0.4193, 0.1051, 0.0418, 0.0342, 0.0232]"
4,/content/drive/MyDrive/SFCT_images/STC_0181.JPG,USA,"[{'category': '1', 'label': 'animal', 'conf': ...",f2efdae9-efb8-48fb-8a91-eccf79ab4ffb;no cv res...,0.8358,classifier,4.0.1a,[f1856211-cfb7-4a5b-9158-c0f72fd09ee6;;;;;;bla...,"[0.8358, 0.0811, 0.0097, 0.0066, 0.0046]"
5,/content/drive/MyDrive/SFCT_images/STC_0187.JPG,USA,"[{'category': '1', 'label': 'animal', 'conf': ...",aaf3b049-36e6-46dd-9a07-8a580e9618b7;mammalia;...,0.9851,classifier,4.0.1a,[aaf3b049-36e6-46dd-9a07-8a580e9618b7;mammalia...,"[0.9851, 0.0063, 0.0027, 0.0008, 0.0008]"
6,/content/drive/MyDrive/SFCT_images/STC_0270.JPG,USA,"[{'category': '1', 'label': 'animal', 'conf': ...",e237b8d3-997f-4f3b-8736-8c3958f8182c;aves;pass...,0.7282,classifier,4.0.1a,[e237b8d3-997f-4f3b-8736-8c3958f8182c;aves;pas...,"[0.7282, 0.0698, 0.0464, 0.0369, 0.0244]"
7,/content/drive/MyDrive/SFCT_images/STC_0312.JPG,USA,"[{'category': '1', 'label': 'animal', 'conf': ...",aaf3b049-36e6-46dd-9a07-8a580e9618b7;mammalia;...,0.9783,classifier,4.0.1a,[aaf3b049-36e6-46dd-9a07-8a580e9618b7;mammalia...,"[0.9783, 0.0052, 0.0031, 0.0017, 0.0013]"
8,/content/drive/MyDrive/SFCT_images/STC_0355.JPG,USA,"[{'category': '1', 'label': 'animal', 'conf': ...",eeeb5d26-2a47-4d01-a3de-10b33ec0aee4;mammalia;...,0.7417,classifier+rollup_to_order,4.0.1a,[aaf3b049-36e6-46dd-9a07-8a580e9618b7;mammalia...,"[0.5408, 0.1196, 0.0355, 0.0229, 0.0229]"
9,/content/drive/MyDrive/SFCT_images/STC_0482.JPG,USA,"[{'category': '1', 'label': 'animal', 'conf': ...",febff896-db40-4ac8-bcfe-5bb99a600950;mammalia;...,0.9930,classifier,4.0.1a,[febff896-db40-4ac8-bcfe-5bb99a600950;mammalia...,"[0.993, 0.002, 0.001, 0.0008, 0.0007]"


In [29]:
df['prediction'][0]

'aaf3b049-36e6-46dd-9a07-8a580e9618b7;mammalia;carnivora;canidae;canis;latrans;coyote'

In [30]:
df['classifications.classes'][0]

['aaf3b049-36e6-46dd-9a07-8a580e9618b7;mammalia;carnivora;canidae;canis;latrans;coyote',
 'b1352069-a39c-4a84-a949-60044271c0c1;aves;;;;;bird',
 'ba76d46e-25de-45e2-90a8-bd279b650f7c;mammalia;carnivora;felidae;lynx;rufus;bobcat',
 '9805e578-061a-49c0-8abc-ea135381c162;aves;passeriformes;corvidae;corvus;corax;common raven',
 '15abc3f7-7155-450e-a590-1730ee9a1adb;mammalia;carnivora;canidae;canis;aureus;golden jackal']